In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF
from sklearn.model_selection import train_test_split
from sklearn.gaussian_process.kernels import Matern
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.gaussian_process.kernels import RBF, Matern, RationalQuadratic, ExpSineSquared, DotProduct, ConstantKernel as C
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import TimeSeriesSplit
from sklearn.gaussian_process.kernels import RBF, Matern, RationalQuadratic, ExpSineSquared, WhiteKernel, ConstantKernel, DotProduct

In [8]:
msa_vmt = pd.read_csv('../../../results/historical_VMT_msa_basedon2022.csv',index_col=None) 
msa_vmt = msa_vmt.set_index('Year')
msa_vmt = msa_vmt[::-1]
msa_vmt

# Define new column names
new_column_names = ['New Haven (billion miles)', 'Hartford (billion miles)', 'Bridgeport (billion miles)']

# Assign new column names to the DataFrame
msa_vmt.columns = new_column_names

msa_vmt.index.name = 'year'

#msa_vmt.index = pd.to_datetime(msa_vmt.index, format='%Y')
msa_vmt.index.freq = 'AS'
df=msa_vmt

In [9]:
NHM_data = df['New Haven (billion miles)']
y_NHM =NHM_data.values.reshape(-1, 1)
x_NHM = NHM_data.index.to_numpy().reshape(-1, 1)

x_NHM_df = pd.DataFrame(x_NHM, columns=['Year'])

# # Now you can use iloc for indexing
# split_index_x = round(len(x_NHM_df) * 0.9)
# X_train = x_NHM_df.iloc[:split_index_x].copy()
# X_test = x_NHM_df.iloc[split_index_x:].copy()

y_NHM_df = pd.DataFrame(y_NHM, columns=['New Haven (billion miles)'])

# Now you can use iloc for indexing
# split_index_y = round(len(y_NHM_df) * 0.9)
# Y_train = y_NHM_df.iloc[:split_index_y].copy()
# Y_test = y_NHM_df.iloc[split_index_y:].copy()

In [11]:
# Define kernels with a constant kernel to allow for parameter setting
kernels = [
    C(3.0) * RBF(length_scale=2.0),
    C(3.0) * Matern(length_scale=2.0, nu=1.5),
    C(3.0) * RationalQuadratic(length_scale=2.0, alpha=0.1)]
    #C(3.0) * ConstantKernel(constant_value=1.0),
    #C(3.0) * DotProduct(sigma_0=1.0),
    #C(3.0) * ExpSineSquared(length_scale=2.0, periodicity=1.0),
    #C(3.0) * RBF(length_scale=1.0) + WhiteKernel(noise_level=0.1),
    #C(3.0) * DotProduct() + C(1.0) * RBF(length_scale=2.0)]

# Time series split for cross-validation
tscv = TimeSeriesSplit(n_splits=5)

# Assuming X and y are already defined
X = x_NHM
y = y_NHM

# List to store the best scores and corresponding kernels and parameters
results = []

# Logarithmically spaced values for length scale and alpha
length_scales = np.logspace(-2, 2, 5)
alphas = np.logspace(-2, 2, 5)
nus = [0.5, 1.5, 2.5]

# length_scales = np.array([-0.1,0,1,2,4,10])
# alphas = np.array([-0.1,0,1,2,4,10])

# Define the mean absolute percentage error function
def mean_absolute_percentage_error(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / (y_true + 1e-10))) * 100

for kernel in kernels:
    for length_scale in length_scales:
        for alpha in alphas:
            for nu in nus:
                try:
                    kernel_copy = kernel.clone_with_theta(kernel.theta)
                    # Set the length scale parameter correctly for different kernels
                    if hasattr(kernel.k2, 'length_scale'):
                        kernel.k2.length_scale = length_scale
                    elif hasattr(kernel.k1, 'length_scale'):
                        kernel.k1.length_scale = length_scale

                    # Set the alpha parameter if the kernel is RationalQuadratic
                    if isinstance(kernel.k2, RationalQuadratic):
                        kernel.k2.alpha = alpha
                    elif isinstance(kernel.k1, RationalQuadratic):
                        kernel.k1.alpha = alpha
    
    # Set the nu parameter if the kernel is Matern
                    if isinstance(kernel_copy.k2, Matern):
                        kernel_copy.k2.nu = nu
                    elif isinstance(kernel_copy.k1, Matern):
                        kernel_copy.k1.nu = nu

                    rmse_scores = []
                    mae_scores = []
                    mape_scores = []
                    for train_index, test_index in tscv.split(X):
                        X_train, X_test = X[train_index], X[test_index]
                        y_train, y_test = y[train_index], y[test_index]

                        gp = GaussianProcessRegressor(kernel=kernel_copy, n_restarts_optimizer=10)
                        gp.fit(X_train, y_train)
                        y_pred, _ = gp.predict(X_test, return_std=True)

                        # Compute the metrics
                        rmse = np.sqrt(mean_squared_error(y_test, y_pred))
                        mae = mean_absolute_error(y_test, y_pred)
                        mape = mean_absolute_percentage_error(y_test, y_pred)

                        rmse_scores.append(rmse)
                        mae_scores.append(mae)
                        mape_scores.append(mape)

                    mean_rmse = np.mean(rmse_scores)
                    mean_mae = np.mean(mae_scores)
                    mean_mape = np.mean(mape_scores)

                    results.append((mean_rmse, mean_mae, mean_mape, kernel_copy, kernel_copy.get_params()))
                except Exception as e:
                    print(f"An error occurred: {e}")

# Sort results by the RMSE score
results.sort(key=lambda x: x[0])          
            
            
            
# # Sort results by the RMSE score
# results.sort(key=lambda x: x[0])

# # Print the top 10 results
# print("Top 10 Kernel Configurations:")
# for i, (rmse, mae, mape, kernel, params) in enumerate(results[:10]):
#     print(f"Rank {i+1}:")
#     print(f"  RMSE: {rmse}")
#     print(f"  MAE: {mae}")
#     print(f"  MAPE: {mape}")
#     print(f"  Kernel: {kernel}")
#     print(f"  Parameters: {params}")
#     print()
    
# # Sort results by the RMSE score
# results.sort(key=lambda x: x[0])

# Categorize results based on kernel types
categorized_results = {
    'RBF': [],
    'Matern': [],
    'RationalQuadratic': [],
    'ConstantKernel': [],
    'DotProduct': [],
    'ExpSineSquared': [],
    'RBF+WhiteKernel': [],
    'DotProduct+RBF': []
}

for result in results:
    kernel = result[3]
    if isinstance(kernel.k1, ConstantKernel) and isinstance(kernel.k2, RBF):
        categorized_results['RBF'].append(result)
    elif isinstance(kernel.k1, ConstantKernel) and isinstance(kernel.k2, Matern):
        categorized_results['Matern'].append(result)
    elif isinstance(kernel.k1, ConstantKernel) and isinstance(kernel.k2, RationalQuadratic):
        categorized_results['RationalQuadratic'].append(result)
    elif isinstance(kernel.k1, ConstantKernel) and isinstance(kernel.k2, ConstantKernel):
        categorized_results['ConstantKernel'].append(result)
    elif isinstance(kernel.k1, ConstantKernel) and isinstance(kernel.k2, DotProduct):
        categorized_results['DotProduct'].append(result)
    elif isinstance(kernel.k1, ConstantKernel) and isinstance(kernel.k2, ExpSineSquared):
        categorized_results['ExpSineSquared'].append(result)
    elif isinstance(kernel.k2, WhiteKernel) and isinstance(kernel.k1.k2, RBF):
        categorized_results['RBF+WhiteKernel'].append(result)
    elif isinstance(kernel.k1, DotProduct) and isinstance(kernel.k2, RBF):
        categorized_results['DotProduct+RBF'].append(result)

# Sort and select the top 5 for each kernel type
for kernel_type in categorized_results:
    categorized_results[kernel_type].sort(key=lambda x: x[0])  # Sort by RMSE
    categorized_results[kernel_type] = categorized_results[kernel_type][:5]  # Select top 5

# Function to create a table from results
def create_table(results, kernel_type):
    table_data = []
    for result in results:
        mean_rmse, mean_mae, mean_mape, kernel, params = result
        row = {
            'Kernel Type': kernel_type,
            'RMSE': mean_rmse,
            'MAE': mean_mae,
            'MAPE': mean_mape,
            'Length Scale': params.get('k2__length_scale') or params.get('k1__length_scale', 'N/A'),
            'Alpha': params.get('k2__alpha') or params.get('k1__alpha', 'N/A'),
            'Nu': params.get('k2__nu') or params.get('k1__nu', 'N/A')
        }
        table_data.append(row)
    return table_data

# Create tables for each kernel type
tables = {kernel_type: create_table(results, kernel_type) for kernel_type, results in categorized_results.items()}

# Print tables
for kernel_type, table in tables.items():
    if table:  # Only print if there are results
        print(f"Top 5 Configurations for {kernel_type} Kernel")
        print(tabulate(table, headers='keys', tablefmt='grid'))
        print()

C:\Users\peiyaozhao\AppData\Local\anaconda3\Lib\site-packages\sklearn\gaussian_process\kernels.py:419: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__length_scale is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
C:\Users\peiyaozhao\AppData\Local\anaconda3\Lib\site-packages\sklearn\gaussian_process\kernels.py:419: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__length_scale is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
C:\Users\peiyaozhao\AppData\Local\anaconda3\Lib\site-packages\sklearn\gaussian_process\kernels.py:419: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__length_scale is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
C:\Users\peiyaozhao\AppData\Loc

Top 5 Configurations for RBF Kernel


NameError: name 'tabulate' is not defined

In [12]:
from tabulate import tabulate

# Print tables
for kernel_type, table in tables.items():
    if table:  # Only print if there are results
        print(f"Top 5 Configurations for {kernel_type} Kernel")
        print(tabulate(table, headers='keys', tablefmt='grid'))
        print()

Top 5 Configurations for RBF Kernel
+---------------+---------+----------+---------+----------------+---------+------+
| Kernel Type   |    RMSE |      MAE |    MAPE |   Length Scale | Alpha   |   Nu |
+===============+=========+==========+=========+================+=========+======+
| RBF           | 0.40671 | 0.350434 | 5.58714 |          10    | N/A     |  0.5 |
+---------------+---------+----------+---------+----------------+---------+------+
| RBF           | 0.40671 | 0.350434 | 5.58714 |         100    | N/A     |  0.5 |
+---------------+---------+----------+---------+----------------+---------+------+
| RBF           | 0.40671 | 0.350434 | 5.58714 |           1    | N/A     |  0.5 |
+---------------+---------+----------+---------+----------------+---------+------+
| RBF           | 0.40671 | 0.350434 | 5.58714 |           0.01 | N/A     |  0.5 |
+---------------+---------+----------+---------+----------------+---------+------+
| RBF           | 0.40671 | 0.350434 | 5.58714 |   

In [ ]:
# Define the best kernel and the corresponding scalar value
best_kernel = results[0][3]
best_params = results[0][4]

# Set the best parameters to the best kernel
for param, value in best_params.items():
    setattr(best_kernel, param, value)

# Train a Gaussian Process Regressor with the best kernel
gp_best_combined = GaussianProcessRegressor(kernel=best_kernel, n_restarts_optimizer=10)
gp_best_combined.fit(X, y)

# Make predictions for the entire dataset using the best combined kernel
y_mean_combined, y_cov_combined = gp_best_combined.predict(X, return_cov=True)

# Define the number of years to forecast
num_years_forecast = 13

# Extend the range of x_NHM to cover the forecast period
x_forecast = np.arange(X.min(), X.max() + num_years_forecast + 1).reshape(-1, 1)

# Make predictions for the forecast period using the best combined kernel
y_mean_forecast_combined, y_cov_forecast_combined = gp_best_combined.predict(x_forecast, return_cov=True)

# Plot the observed data, predicted mean with the best combined kernel, and forecast
plt.figure(figsize=(10, 5))
plt.scatter(X, y, c='b', label='Observed Data')
# plt.plot(X, y_mean_combined, 'k', lw=2, zorder=9, label='Predicted Mean (Best Kernel)')
plt.fill_between(X[:, 0], y_mean_combined - 1.96 * np.sqrt(np.diag(y_cov_combined)),
                 y_mean_combined + 1.96 * np.sqrt(np.diag(y_cov_combined)), alpha=0.8, color='blue', label='95% Confidence Interval')

plt.plot(x_forecast, y_mean_forecast_combined, '--', color='green', label='Forecast')
plt.fill_between(x_forecast[:, 0], y_mean_forecast_combined - 1.96 * np.sqrt(np.diag(y_cov_forecast_combined)),
                 y_mean_forecast_combined + 1.96 * np.sqrt(np.diag(y_cov_forecast_combined)), alpha=0.2, color='green', label='95% Confidence Interval (Forecast)')

plt.xlabel('Year')
plt.ylabel('Value')
plt.legend()
plt.show()

In [13]:
import numpy as np
import pandas as pd
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, Matern, RationalQuadratic, ConstantKernel, DotProduct, ExpSineSquared, WhiteKernel
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import TimeSeriesSplit
from tabulate import tabulate

# Example data (replace these with your actual data)
X = np.random.randn(100, 1)  # Example data
y = np.random.randn(100)  # Example target

# Define kernels with a constant kernel to allow for parameter setting
C = ConstantKernel

kernels = [
    C(3.0) * RBF(length_scale=2.0),
    C(3.0) * Matern(length_scale=2.0, nu=1.5),
    C(3.0) * RationalQuadratic(length_scale=2.0, alpha=0.1),
    C(3.0) * ConstantKernel(constant_value=1.0),
    C(3.0) * DotProduct(sigma_0=1.0),
    C(3.0) * ExpSineSquared(length_scale=2.0, periodicity=1.0),
    C(3.0) * RBF(length_scale=1.0) + WhiteKernel(noise_level=0.1),
    C(3.0) * DotProduct() + C(1.0) * RBF(length_scale=2.0)
]

# Time series split for cross-validation
tscv = TimeSeriesSplit(n_splits=5)

# List to store the best scores and corresponding kernels and parameters
results = []

# Logarithmically spaced values for length scale and alpha
length_scales = np.logspace(-2, 2, 5)
alphas = np.logspace(-2, 2, 5)

# Define the mean absolute percentage error function
def mean_absolute_percentage_error(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / (y_true + 1e-10))) * 100

# Evaluate each kernel with different length scales and alphas
for kernel in kernels:
    for length_scale in length_scales:
        for alpha in alphas:
            try:
                # Create a copy of the kernel to modify
                kernel_copy = kernel.clone_with_theta(kernel.theta)

                # Set the length scale parameter correctly for different kernels
                if hasattr(kernel_copy.k2, 'length_scale'):
                    kernel_copy.k2.length_scale = length_scale
                elif hasattr(kernel_copy.k1, 'length_scale'):
                    kernel_copy.k1.length_scale = length_scale

                # Set the alpha parameter if the kernel is RationalQuadratic
                if isinstance(kernel_copy.k2, RationalQuadratic):
                    kernel_copy.k2.alpha = alpha
                elif isinstance(kernel_copy.k1, RationalQuadratic):
                    kernel_copy.k1.alpha = alpha

                rmse_scores = []
                mae_scores = []
                mape_scores = []
                for train_index, test_index in tscv.split(X):
                    X_train, X_test = X[train_index], X[test_index]
                    y_train, y_test = y[train_index], y[test_index]

                    gp = GaussianProcessRegressor(kernel=kernel_copy, n_restarts_optimizer=10)
                    gp.fit(X_train, y_train)
                    y_pred, _ = gp.predict(X_test, return_std=True)

                    # Compute the metrics
                    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
                    mae = mean_absolute_error(y_test, y_pred)
                    mape = mean_absolute_percentage_error(y_test, y_pred)

                    rmse_scores.append(rmse)
                    mae_scores.append(mae)
                    mape_scores.append(mape)

                mean_rmse = np.mean(rmse_scores)
                mean_mae = np.mean(mae_scores)
                mean_mape = np.mean(mape_scores)

                results.append((mean_rmse, mean_mae, mean_mape, kernel_copy, kernel_copy.get_params()))
            except Exception as e:
                print(f"An error occurred: {e}")

# Sort results by the RMSE score
results.sort(key=lambda x: x[0])

# Categorize results based on kernel types
categorized_results = {
    'RBF': [],
    'Matern': [],
    'RationalQuadratic': [],
    'ConstantKernel': [],
    'DotProduct': [],
    'ExpSineSquared': [],
    'RBF+WhiteKernel': [],
    'DotProduct+RBF': []
}

for result in results:
    kernel = result[3]
    if isinstance(kernel.k1, ConstantKernel) and isinstance(kernel.k2, RBF):
        categorized_results['RBF'].append(result)
    elif isinstance(kernel.k1, ConstantKernel) and isinstance(kernel.k2, Matern):
        categorized_results['Matern'].append(result)
    elif isinstance(kernel.k1, ConstantKernel) and isinstance(kernel.k2, RationalQuadratic):
        categorized_results['RationalQuadratic'].append(result)
    elif isinstance(kernel.k1, ConstantKernel) and isinstance(kernel.k2, ConstantKernel):
        categorized_results['ConstantKernel'].append(result)
    elif isinstance(kernel.k1, ConstantKernel) and isinstance(kernel.k2, DotProduct):
        categorized_results['DotProduct'].append(result)
    elif isinstance(kernel.k1, ConstantKernel) and isinstance(kernel.k2, ExpSineSquared):
        categorized_results['ExpSineSquared'].append(result)
    elif isinstance(kernel.k2, WhiteKernel) and isinstance(kernel.k1.k2, RBF):
        categorized_results['RBF+WhiteKernel'].append(result)
    elif isinstance(kernel.k1, DotProduct) and isinstance(kernel.k2, RBF):
        categorized_results['DotProduct+RBF'].append(result)

# Sort and select the top 5 for each kernel type
for kernel_type in categorized_results:
    categorized_results[kernel_type].sort(key=lambda x: x[0])  # Sort by RMSE
    categorized_results[kernel_type] = categorized_results[kernel_type][:5]  # Select top 5

# Function to create a table from results
def create_table(results, kernel_type):
    table_data = []
    for result in results:
        mean_rmse, mean_mae, mean_mape, kernel, params = result
        row = {
            'Kernel Type': kernel_type,
            'RMSE': mean_rmse,
            'MAE': mean_mae,
            'MAPE': mean_mape,
            'Length Scale': params.get('k2__length_scale') or params.get('k1__length_scale', 'N/A'),
            'Alpha': params.get('k2__alpha') or params.get('k1__alpha', 'N/A')
        }
        table_data.append(row)
    return table_data

# Create tables for each kernel type
tables = {kernel_type: create_table(results, kernel_type) for kernel_type, results in categorized_results.items()}

# Print tables
for kernel_type, table in tables.items():
    if table:  # Only print if there are results
        print(f"Top 5 Configurations for {kernel_type} Kernel")
        print(tabulate(table, headers='keys', tablefmt='grid'))
        print()


C:\Users\peiyaozhao\AppData\Local\anaconda3\Lib\site-packages\sklearn\gaussian_process\kernels.py:419: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__length_scale is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
C:\Users\peiyaozhao\AppData\Local\anaconda3\Lib\site-packages\sklearn\gaussian_process\kernels.py:419: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__length_scale is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
C:\Users\peiyaozhao\AppData\Local\anaconda3\Lib\site-packages\sklearn\gaussian_process\kernels.py:419: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__length_scale is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
C:\Users\peiyaozhao\AppData\Loc

Top 5 Configurations for RBF Kernel
+---------------+----------+----------+--------+----------------+---------+
| Kernel Type   |     RMSE |      MAE |   MAPE |   Length Scale | Alpha   |
+===============+==========+==========+========+================+=========+
| RBF           | 0.930199 | 0.788808 |    100 |           0.01 | N/A     |
+---------------+----------+----------+--------+----------------+---------+
| RBF           | 0.930199 | 0.788808 |    100 |           0.01 | N/A     |
+---------------+----------+----------+--------+----------------+---------+
| RBF           | 0.930199 | 0.788808 |    100 |           0.01 | N/A     |
+---------------+----------+----------+--------+----------------+---------+
| RBF           | 0.930199 | 0.788808 |    100 |           0.01 | N/A     |
+---------------+----------+----------+--------+----------------+---------+
| RBF           | 0.930199 | 0.788808 |    100 |           0.01 | N/A     |
+---------------+----------+----------+--------+----

C:\Users\peiyaozhao\AppData\Local\anaconda3\Lib\site-packages\sklearn\gaussian_process\kernels.py:419: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__k1__constant_value is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
C:\Users\peiyaozhao\AppData\Local\anaconda3\Lib\site-packages\sklearn\gaussian_process\kernels.py:419: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__k2__sigma_0 is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
C:\Users\peiyaozhao\AppData\Local\anaconda3\Lib\site-packages\sklearn\gaussian_process\kernels.py:419: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__k2__length_scale is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
